# Exercise 1 - Building a 0-shot classifier with Outlines

In this notebook we will take our first look at how to use [Outlines](https://github.com/outlines-dev/outlines). Most of the code is written for you, but you will be required to implement small snippets of code to make sure you understand the key concepts being discussed.




In [ ]:
import json
import outlines
import torch
from transformers import AutoTokenizer
from textwrap import dedent

Here we load our model using `outlines.models.transformers`.

**Note**: If you are not using an Apple Silicon device, you should change `mps` to either `cuda` if you have an NVIDIA GPU or `cpu` otherwise.

In [ ]:
model_name = "Qwen/Qwen2-0.5B-Instruct"
model = outlines.models.transformers(
    model_name,
    device='mps',
    model_kwargs={
        'torch_dtype': torch.bfloat16,
        'trust_remote_code': True
    })
tokenizer = AutoTokenizer.from_pretrained(model_name)

## The Task: Predicting Department

In this example we'll be processing messages that have received involving user complaints, here is the data loaded with a few examples:

In [ ]:
with open("../examples.json",'r') as fin:
    complaint_data = json.loads(fin.read())
complaint_data[0:3]

As you can see, there are departments associated with each complaint. What we want to do is build a **zero-shot** classifier to predcit the department given the content of the `message`. Here are the departments in our example:

In [ ]:
departments = ["clothing","electronics","kitchen","automotive"]

Zero-shot classifiers are great use cases for LLMs because they allow us to try to classify data with *zero* example cases. This is particularly helpful when we don't have any existing data to train a traditional machine learning classifier on. For the purpose of this workshop we're using defaulting to the small but not very powerful `Qwen/Qwen2-0.5B-Instrct` model. This is to ensure this model runs reasonably well on most laptops. This model performs relatively poorly on the task, but if we were to upgrade to something like `Phi-3-medium` we can get 92% accuracy on this task *with no traning examples required!*.

### Our Prompt

We are going to be using an `instruct` model which means we can use a *instruction* or *chat-based* style of prompting. The function below takes our complaint and formats it into a prompt to send to the LLM. Note the use of `tokenizer.apply_chat_template`. Each instruct LLM has a slightly different format for it's instruction prompts, but using `apply_chat_template` makes it easy to generalize.

In [ ]:
def create_prompt(complaint):
    prompt_messages = [
        {
            "role": "system",
            "content": "You are as agent designed to help label complaints."
        },
        {
        "role": "user",
        "content": dedent("""
        I'm going to provide you with a consumer complaint to analyze.
        The complaint is going to be regarding a product from one of our
        departments. Here is the list of departments:
            - "clothing"
            - "electronics"
            - "kitchen"
            - "automotive"
        Please reply with *only* the name of the department.
        """)
    },{
        "role": "assistant",
        "content": "I understand and will only answer with the department name"
    },{
        "role": "user",
        "content": f"Great! Here is the complaint: {complaint['message']}"
    }
                       
                      ]
    prompt = tokenizer.apply_chat_template(prompt_messages, tokenize=False)
    return prompt

This is how the first complaint looks when rendered as a prompt for our model:

In [ ]:
create_prompt(complaint_data[0])

## Unstructured Generation

To demonstrate the value of *structured* generation we'll first start with an *unstructured* example. We can still use Outlines for this! In this case we'll use `outlines.generate.text` to create a text generator using our LLMs.

In [ ]:
generator = outlines.generate.text(model)

Next we iterate through a few examples to see what our unstructured response looks like:

In [ ]:
for complaint in complaint_data[0:4]:
    prompt = create_prompt(complaint)
    result = generator(prompt, max_tokens=12)
    print(f"LLM: {result} Actual: {complaint['department']}")

As you can see the results are a bit of mess! In this case we'll need to do a bit of additional parse to out the actual answer (if there is one).

Using Structured Generation with Outlines we no longer have to worry about this.

## Structured Generation

This first exercise is quite simple, you are going to use `outlines.generate.choice` to create a structured generator that will product only structured text. Replace the:

```
generator_struct = None
```

With the correct code to solve this problem with `choice`. This task should be very easy, but it's a good first start at using Outlines.

In [ ]:
correct = 0
# Exercise 1 is just to fill in this code.
generator_struct = None
for complaint in complaint_data:
    prompt = create_prompt(complaint)
    result = generator_struct(prompt)
    if result == complaint['department']:
        correct += 1
    print(f"LLM: {result} Actual: {complaint['department']}")
print(correct/len(complaint_data))

Depending on the underlying LLM you use, you may get dramatically different performance but in all cases you should find that the model *always* produces just a single, valid department name as an option.